# Introduction 
 The recent paideia project

In [8]:
import  pandas as pd
import  os
from    datetime import datetime
from    tqdm import tqdm
import  regex as re
import  time
import  numpy as np
import  random
import  pyperclip as py
# import  PyPDF2
import  openai 
import  regex as re
import  random 
import  os

import  utilities as my

from    dotenv import load_dotenv
load_dotenv()
openai.api_type     = os.getenv('OPENAI_TYPE')
openai.api_base     = os.getenv('OPENAI_BASE')
openai.api_version  = os.getenv('OPENAI_VERSION')
openai.api_key      = os.getenv('OPENAI_KEY')

import tiktoken

def get_embedding(text):
  # Note how this function assumes you already set your Open AI key!
    result = openai.Embedding.create(
      engine = 'text-embedding-ada-002',
      model='text-embedding-ada-002',
      input=text
    )
    return result["data"][0]["embedding"]

In [9]:
def get_embedding(text):
  # Note how this function assumes you already set your Open AI key!
    result = openai.Embedding.create(
      engine = 'text-embedding-ada-002',
      model='text-embedding-ada-002',
      input=text
    )
    return result["data"][0]["embedding"]
  
def summary(author,text,title, written_date):
    text = 'In their article {author}, writes with the title: "{title}" which was published on {written_date}. The following: \n {text}'.format(author=author,text=text,title=title,written_date=written_date)
    return text 


In [28]:
def new_func():
    # now lets first make a script which takes the clipboard and converts it into a list of strings
    # the first line is the title
    # the second line is the author
    # the third line is the date
    # the fourth line is the url
    # from there the rest is text
    # the text is split by the new line character
    # then if the line contains less than 20 characters it is removed
    # then we create a dataframe with the title, author, date, and text
    # we then save that to a csv file which is called master, which will store all the articles 
    # lets start by loading in the clipboard
    # we can use pyperclip for this
    x = py.paste()

    # now lets split the text by the new line character
    x = x.split('\r\n')

    runable = True

    title = x[0]
    author = x[1]
    date = x[2]
    url = x[3]

    if len(date.split('-')) < 2:
        runable = False
    
    # advertise sentences
    advertisements = ['Subscribe here to ',
                  ]

    if runable:
    # now lets remove all the lines which have less than 20 characters
        text = x[4:]

    # now lets create a dataframe with the title, author, date, and text
        df = pd.DataFrame(columns=['title', 'author', 'date', 'url', 'text'])
        df['text'] = text

        df['title'] = title
        df['author'] = author
        df['date'] = date
        df['url'] = url

    # now lets get the length of the text column in a sepearet column
        df['length'] = df['text'].apply(lambda x: len(x))

        def num_tokens_from_string(string, encoding_name):
            """Returns the number of tokens in a text string."""
            encoding = tiktoken.get_encoding(encoding_name)
            num_tokens = len(encoding.encode(string))
            return num_tokens

        df['token_count'] = df['text'].apply(lambda text: num_tokens_from_string(text,'cl100k_base'))

    # now lets go through the dataframe and join on the text cells until they reach a sum of the token count of 200
    # then we will add that to a list and then create a new dataframe with the text and the token count

        new_strings = []

        current_tokens = 0

        y = ''

        for i in range(len(df)):
            x = df.at[i, 'token_count']
        
            if current_tokens + x < 300:
                y += df.at[i, 'text']
                current_tokens += x
            else:
                new_strings.append(y)
                y = ''
                current_tokens = 0
        
    # now lets overwrite the df with the new_strings values but the same title, author, date, and url
        df = pd.DataFrame(columns=['title', 'author', 'date', 'url', 'text'])
        df['text'] = new_strings
        df['author'] = author
        df['date'] = date
        df['url'] = url
        df['title'] = title

        new_line = [title, author, date, url, '[Insert Summary Here]']
        df.loc[len(df)] = new_line

        df['one_line'] = ''
        df['question'] = ''
    
        df['summary'] = df.apply(lambda x: summary(x['author'],x['text'],x['title'],x['date']),axis=1)
    
        df['embedding'] = df['summary'].apply(lambda x: get_embedding(x))
    else:
        print('The date is not in the correct format')
        df = ''
    
    
    return df

df = new_func()

In [29]:
df

,title,author,date,url,text,one_line,question,summary,embedding
0,This Is The Simple Way To Achieve Your Goals: ...,Erik Barker,01-12-2023,https://bakadesuyo.com/2023/12/strategy/,Strategy. It’s treated with a reverence usuall...,,,"In their article Erik Barker, writes with the ...","[-0.00908722449094057, 0.003831280628219247, 0..."
1,This Is The Simple Way To Achieve Your Goals: ...,Erik Barker,01-12-2023,https://bakadesuyo.com/2023/12/strategy/,But the true first step is knowing what bad st...,,,"In their article Erik Barker, writes with the ...","[-0.012964476831257343, 0.009832119569182396, ..."
2,This Is The Simple Way To Achieve Your Goals: ...,Erik Barker,01-12-2023,https://bakadesuyo.com/2023/12/strategy/,So what is strategy?The Definition Of Strategy...,,,"In their article Erik Barker, writes with the ...","[-0.009631097316741943, 0.003807718399912119, ..."
3,This Is The Simple Way To Achieve Your Goals: ...,Erik Barker,01-12-2023,https://bakadesuyo.com/2023/12/strategy/,You need to think about obstacles. The stuff t...,,,"In their article Erik Barker, writes with the ...","[-0.0018082158640027046, 0.008606971241533756,..."
4,This Is The Simple Way To Achieve Your Goals: ...,Erik Barker,01-12-2023,https://bakadesuyo.com/2023/12/strategy/,Now “focused and simple” sounds good but this ...,,,"In their article Erik Barker, writes with the ...","[-0.020438166335225105, 0.00015029303904157132..."
5,This Is The Simple Way To Achieve Your Goals: ...,Erik Barker,01-12-2023,https://bakadesuyo.com/2023/12/strategy/,"So set a “proximate objective.” Yes, that soun...",,,"In their article Erik Barker, writes with the ...","[-0.018216855823993683, -0.01605622097849846, ..."
6,This Is The Simple Way To Achieve Your Goals: ...,Erik Barker,01-12-2023,https://bakadesuyo.com/2023/12/strategy/,Guiding Policy: How can you apply your str...,,,"In their article Erik Barker, writes with the ...","[-0.017031535506248474, -0.006848321296274662,..."
7,This Is The Simple Way To Achieve Your Goals: ...,Erik Barker,01-12-2023,https://bakadesuyo.com/2023/12/strategy/,[Insert Summary Here],,,"In their article Erik Barker, writes with the ...","[-0.014179251156747341, 0.005765299312770367, ..."


In [30]:
master = pd.read_excel('master.xlsx')

master.tail(3)

,title,author,date,url,text,one_line,question,summary,token_count,embedding,prompt_similarity
59,This Is How To Make Emotionally Intelligent Fr...,Erik Barker,01-11-2023,https://bakadesuyo.com/2023/11/listening/,"Sure, asking someone, “What’s the last thing y...",NaN,NaN,"In their article Erik Barker, writes with the ...",NaN,"[-0.01197544950991869, 0.011955860070884228, 0...",NaN
60,This Is How To Make Emotionally Intelligent Fr...,Erik Barker,01-11-2023,https://bakadesuyo.com/2023/11/listening/,Don’t Be A Topper: Nobody likes these peop...,NaN,NaN,"In their article Erik Barker, writes with the ...",NaN,"[-0.0040417020209133625, 0.011361939832568169,...",NaN
61,This Is How To Make Emotionally Intelligent Fr...,Erik Barker,01-11-2023,https://bakadesuyo.com/2023/11/listening/,[Insert Summary Here],NaN,NaN,"In their article Erik Barker, writes with the ...",NaN,"[-0.011674759909510612, 0.002836287720128894, ...",NaN


In [31]:
master = pd.read_excel('master.xlsx')

# now lets concatenate the df to the master
master = pd.concat([master, df], ignore_index=True)

master.to_excel('master.xlsx', index=False)


In [32]:
master

,title,author,date,url,text,one_line,question,summary,token_count,embedding,prompt_similarity
0,"Slow-burn AI: When augmentation, not automatio...",Sangeet Paul Choudary,03-12-2023,https://platforms.substack.com/p/slow-burn-ai-...,"You’d be quick to conclude then, that, those w...",Sangeet Paul Choudary argues that the real thr...,NaN,"In their article Sangeet Paul Choudary, writes...",214.0,"[-0.011768677271902561, -0.02153008058667183, ...",0.763318
1,"Slow-burn AI: When augmentation, not automatio...",Sangeet Paul Choudary,03-12-2023,https://platforms.substack.com/p/slow-burn-ai-...,This issue right here is Part 1 of a three-par...,"Sangeet Paul Choudary's article ""Slow-burn AI:...",NaN,"In their article Sangeet Paul Choudary, writes...",226.0,"[-0.009319336153566837, -0.016895689070224762,...",0.755640
2,"Slow-burn AI: When augmentation, not automatio...",Sangeet Paul Choudary,03-12-2023,https://platforms.substack.com/p/slow-burn-ai-...,"Up until the start of the twentieth century, a...",Sangeet Paul Choudary argues that augmentation...,NaN,"In their article Sangeet Paul Choudary, writes...",222.0,"[-0.008647583425045013, -0.013699556700885296,...",0.734889
3,"Slow-burn AI: When augmentation, not automatio...",Sangeet Paul Choudary,03-12-2023,https://platforms.substack.com/p/slow-burn-ai-...,When technology augments skilled work and enab...,"Sangeet Paul Choudary's article ""Slow-burn AI:...",NaN,"In their article Sangeet Paul Choudary, writes...",209.0,"[-0.014073953963816166, -0.016499046236276627,...",0.752511
4,"Slow-burn AI: When augmentation, not automatio...",Sangeet Paul Choudary,03-12-2023,https://platforms.substack.com/p/slow-burn-ai-...,"This is an early study, possibly with many cav...","Sangeet Paul Choudary's article ""Slow-burn AI:...",NaN,"In their article Sangeet Paul Choudary, writes...",236.0,"[-0.013471231795847416, -0.02042117714881897, ...",0.753413
...,...,...,...,...,...,...,...,...,...,...,...
65,This Is The Simple Way To Achieve Your Goals: ...,Erik Barker,01-12-2023,https://bakadesuyo.com/2023/12/strategy/,You need to think about obstacles. The stuff t...,,,"In their article Erik Barker, writes with the ...",NaN,"[-0.0018082158640027046, 0.008606971241533756,...",NaN
66,This Is The Simple Way To Achieve Your Goals: ...,Erik Barker,01-12-2023,https://bakadesuyo.com/2023/12/strategy/,Now “focused and simple” sounds good but this ...,,,"In their article Erik Barker, writes with the ...",NaN,"[-0.020438166335225105, 0.00015029303904157132...",NaN
67,This Is The Simple Way To Achieve Your Goals: ...,Erik Barker,01-12-2023,https://bakadesuyo.com/2023/12/strategy/,"So set a “proximate objective.” Yes, that soun...",,,"In their article Erik Barker, writes with the ...",NaN,"[-0.018216855823993683, -0.01605622097849846, ...",NaN
68,This Is The Simple Way To Achieve Your Goals: ...,Erik Barker,01-12-2023,https://bakadesuyo.com/2023/12/strategy/,Guiding Policy: How can you apply your str...,,,"In their article Erik Barker, writes with the ...",NaN,"[-0.017031535506248474, -0.006848321296274662,...",NaN
